Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

Sentence Transformers


In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:06<00:00, 64.1MB/s]


In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

sentence_embeddings = model.encode(sentences, show_progress_bar=True, num_workers=4, device = 'cuda')
print("First_Sentence:", sentences[0])
print(sentence_embeddings[0]) 

In [ ]:
!ls

drive  sample_data


In [ ]:
import numpy as np
import pandas as pd
import pickle
import keras
from array import array
from numpy import *
import time

Reading Twitter Data File

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Covid19/cleaned_650k_tweets.csv')

In [ ]:
dataset.head()

,TWEETS,CLASS
0,is upset that he can t update his facebook by ...,0
1,spring break in plain city it s snowing,0
2,body of missing northern calif girl found poli...,0
3,omg i just moisturised and my legs are burning...,0
4,my reply is no,0


In [ ]:
print('Length of dataset is: ', len(dataset))

Length of dataset is:  639057


In [ ]:
dataset['TWEETS'][0]
#type(dataset['TWEETS'][0])

'is upset that he can t update his facebook by texting it and might cry as a result school today also blah'

Cleaning Dataset


In [ ]:
import re
corpus = []
for i in range(len(dataset)):     #  for i in range(0, 1000):
  text = dataset['TWEETS'][i]
  #text = text.lower()
  text = re.sub('(@\w+)', '', text)  # Removing @ followed by words i.e. usernames
  text = re.sub('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)   # Removing hyperlinks
  text = re.sub('\s+', ' ', text) # Removing multiple spaces with single space
  text = text.strip()
  dataset['TWEETS'][i] = text 
  print('The current sample is: ', i)
print('COMPLETED')




In [ ]:
dataset.head()

,TWEETS,CLASS
0,is upset that he can t update his facebook by ...,0
1,spring break in plain city it s snowing,0
2,body of missing northern calif girl found poli...,0
3,omg i just moisturised and my legs are burning...,0
4,my reply is no,0


In [ ]:
#Converting corpus_df to list format as BERT model taked list of sentences as input
corpus_list = dataset['TWEETS'].values.tolist()
#corpus_list = sub_dataset['TWEETS'].values.tolist()

In [ ]:
corpus_list

In [ ]:
len(corpus_list)

639057

In [ ]:
!nvidia-smi

Fri Dec 11 14:31:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    38W / 300W |   1831MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

BERT EMBEDDINGS

In [ ]:
# Calculating BERT Sentence embeddings
start_time = time.time()
print('start_time_is: ', start_time)

sentence_embeddings = model.encode(corpus_list, show_progress_bar=True, device = 'cuda')
print('Time taken %.2f seconds ----' % (time.time() - start_time))
print('Time taken %.2f Minutes ----' % (((time.time() - start_time)/60)))


start_time_is:  1607697113.8448932



Time taken 549.45 seconds ----
Time taken 9.16 Minutes ----


In [ ]:
print('Length of sentence embeddings is: ', len(sentence_embeddings))
print('Type of sentence embeddings is:', type(sentence_embeddings))

Length of sentence embeddings is:  639057
Type of sentence embeddings is: <class 'numpy.ndarray'>


In [ ]:
print(sentence_embeddings[0])

[-2.86479771e-01  4.48571563e-01  1.26767015e+00 -6.35097921e-02
 -5.12126267e-01 -3.61853093e-01  1.89716268e+00  2.87906677e-01
  5.23605824e-01 -5.23396432e-02 -1.07788658e+00  1.63251191e-01
  6.11931443e-01  1.77009925e-01 -9.68112767e-01  3.23262125e-01
  3.58557552e-01  1.23846486e-01 -4.39199150e-01 -3.06572646e-01
 -3.13164711e-01 -8.89964819e-01  1.08413351e+00 -3.46566468e-01
  1.26310587e+00  4.59897876e-01 -5.00520885e-01  5.20028651e-01
 -9.37702656e-01 -1.75469264e-01  5.58186471e-02  7.54925251e-01
  3.00782979e-01  4.93758649e-01 -3.22590798e-01  4.35005337e-01
  6.72415257e-01 -4.83727485e-01  3.36178422e-01 -3.02076101e-01
 -3.03052902e-01  4.45054710e-01 -8.20289612e-01 -8.17446887e-01
  7.94527307e-02 -5.07662177e-01  1.18293989e+00 -3.71640086e-01
  6.74301207e-01 -8.80671978e-01 -1.49533421e-01  8.07880163e-01
  1.70285869e+00  4.53894019e-01 -5.13536751e-01 -7.90246964e-01
  3.52444381e-01 -1.57253194e+00 -1.80245012e-01 -8.10235500e-01
 -1.06157553e+00  8.81555

In [ ]:

y = dataset.iloc[:, 1].values

print('Len of y is: ', len(y))
print(y)

Len of y is:  639057
[0 0 0 ... 4 4 4]


In [ ]:
# Converting the generated 'sentence_embeddings' back to dataframe
df_embedding = pd.DataFrame(sentence_embeddings)
print('len of df_embedding: ', len(df_embedding))

len of df_embedding:  639057


In [ ]:
y= dataset['CLASS']

In [ ]:

df_embedding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.286480,0.448572,1.267670,-0.063510,-0.512126,-0.361853,1.897163,0.287907,0.523606,-0.052340,-1.077887,0.163251,0.611931,0.177010,-0.968113,0.323262,0.358558,0.123846,-0.439199,-0.306573,-0.313165,-0.889965,1.084134,-0.346566,1.263106,0.459898,-0.500521,0.520029,-0.937703,-0.175469,0.055819,0.754925,0.300783,0.493759,-0.322591,0.435005,0.672415,-0.483727,0.336178,-0.302076,...,0.456394,0.315260,0.631613,-0.025354,0.196037,-0.299894,0.038038,0.448541,0.859714,-1.017923,-0.229990,-0.611187,0.566558,0.731689,0.086210,-1.038726,0.125580,0.133537,0.401990,-0.420222,-0.208419,0.049626,0.251064,0.130167,-0.227989,1.242335,0.074684,-0.346103,-0.315580,-0.391937,-0.082598,0.091360,-0.721052,-1.907303,0.116018,-1.068429,-0.169181,0.765449,-0.428248,0.000054
1,-0.258666,0.347734,1.750804,0.744646,0.156363,0.059703,0.606871,1.274236,-0.051907,-0.414460,-0.900400,-1.083901,0.238065,0.040403,0.185373,0.233348,-0.223842,-0.417781,0.972869,-0.270457,-0.038910,-1.044466,0.511643,-0.243701,0.872497,-0.779433,0.340661,0.380800,-0.721053,0.327722,-0.943484,0.801839,0.370747,-0.736390,0.492543,1.436235,-1.465481,0.170475,-0.266959,-0.868216,...,-0.946630,0.280665,0.495754,-0.412071,0.190216,-0.197012,-0.586232,-0.148826,0.311156,-0.298051,-0.389379,-1.686341,-1.592549,0.704259,0.778034,0.850522,0.285663,0.283842,-0.357595,-0.127402,-0.374123,-0.541077,0.436833,0.087912,-0.209567,1.176815,-0.177077,0.422629,-0.575461,-0.475836,-0.815868,-0.885349,-0.129828,0.342767,-0.074613,-0.007231,0.095131,-0.297645,-0.237426,0.405776
2,0.068400,0.183420,-0.214393,0.162653,0.577132,0.442029,0.764438,-0.231416,0.047675,0.447656,0.244569,-0.412664,0.697799,0.426231,-0.888598,-0.014143,-0.230498,0.307927,0.393809,-0.087685,0.218682,-0.956841,0.148737,-0.326993,0.546378,1.332277,-0.055264,-0.255176,-1.306164,0.654879,-0.712077,0.094668,0.096023,-0.113867,0.053256,0.106918,-0.112052,0.227721,-0.690115,1.005142,...,-0.457578,-0.601099,-0.189601,-0.019634,-0.527271,-0.348555,0.013853,0.217463,0.498457,-0.234963,-0.239371,-0.890102,-0.499332,0.441001,0.115897,-1.437539,-0.679651,-0.743852,0.168532,-0.309895,0.265426,0.508672,0.518088,0.438903,0.816456,1.100221,-0.492533,-0.254384,-0.858149,-0.775391,-0.754896,-0.106961,-0.813766,-0.573372,0.283298,-1.146841,-0.188256,-0.448757,-0.169211,-0.402393
3,0.188355,0.749234,1.241174,-0.022515,-0.279121,-0.386572,1.099895,0.240622,0.614666,-0.238493,-0.190334,0.773524,0.447755,0.075109,0.457645,0.740499,-0.564793,-0.748223,-0.005571,-0.381286,0.149510,-0.817235,0.164348,0.037957,0.652394,0.676520,-0.199341,-0.201901,-1.347610,0.141949,0.251515,-0.318091,0.188833,-0.410054,0.170793,0.711227,0.472304,-0.038543,0.390866,-0.200143,...,-0.406574,-0.597680,0.496792,-0.099677,0.001599,-0.403225,0.212363,0.529139,0.696017,-0.658609,-0.574746,0.728369,0.420841,-0.083178,0.094958,-0.814671,-0.469056,0.236358,-0.311130,0.489052,0.181727,-0.319935,-0.147488,-0.030966,-0.183563,0.614271,0.485601,0.107692,-0.957782,-0.018880,0.304408,-0.002369,-0.649310,-2.455173,0.253222,-0.307889,-0.088449,-0.014566,0.152551,0.669754
4,0.212814,0.028354,2.198674,-0.266557,-0.021838,1.170687,0.923018,0.613718,0.008676,0.054019,-1.087324,0.586673,0.272260,0.487963,1.665598,0.155764,-0.577824,-0.609392,-0.021663,-0.091208,-0.054415,0.226796,0.252040,-1.596501,-0.688963,-1.063881,0.012446,-1.601381,-0.698446,0.426477,0.175979,0.685706,0.238747,0.318971,0.546566,0.163142,0.050185,-0.311229,-0.150162,-0.189366,...,-0.456599,-0.937988,-0.446278,0.615909,-0.336407,-0.202670,0.917652,0.180642,0.416239,-0.169856,-0.051804,0.151931,0.108152,0.935536,0.756829,0.140368,0.269411,-0.458251,-0.179566,0.204512,-0.525036,0.551151,1.038969,0.900104,0.064971,-0.623618,-0.379829,-0.745131,-0.930456,-0.3

In [ ]:
df_embedding.to_csv("/content/drive/MyDrive/Covid19/covid_data_with_embeddings_191644.csv")
print('DONE!')

DONE!


 save Bert Embeddings Model to disk

In [ ]:
# save Bert Embeddings Model to disk
filename = '/content/drive/MyDrive/Covid19/covid_data_with_embeddings_191644_NO2.sav'
pickle.dump(df_embedding, open(filename, 'wb'))
print('Model saved')

Model saved


Resume point, as Bert Embeddings Model has been saved 

In [ ]:
import numpy as np
import pandas as pd
import pickle
import keras
from array import array
from numpy import *
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load the Bert Embeddiings from disk
filename = '/content/drive/MyDrive/Covid19/covid_data_with_embeddings_191644.sav'
df_embedding = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)
print('embeddings loaded')


embeddings loaded


In [ ]:
y

0         0
1         0
2         0
3         0
4         0
         ..
191289    4
191290    4
191291    4
191292    4
191293    4
Name: CLASS, Length: 191294, dtype: int64

In [ ]:
df_embedding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.286480,0.448572,1.267670,-0.063510,-0.512126,-0.361853,1.897163,0.287907,0.523606,-0.052340,-1.077887,0.163251,0.611931,0.177010,-0.968113,0.323262,0.358558,0.123846,-0.439199,-0.306573,-0.313165,-0.889965,1.084134,-0.346566,1.263106,0.459898,-0.500521,0.520029,-0.937703,-0.175469,0.055819,0.754925,0.300783,0.493759,-0.322591,0.435005,0.672415,-0.483727,0.336178,-0.302076,...,0.456394,0.315260,0.631613,-0.025354,0.196037,-0.299894,0.038038,0.448541,0.859714,-1.017923,-0.229990,-0.611187,0.566558,0.731689,0.086210,-1.038726,0.125580,0.133537,0.401990,-0.420222,-0.208419,0.049626,0.251064,0.130167,-0.227989,1.242335,0.074684,-0.346103,-0.315580,-0.391937,-0.082598,0.091360,-0.721052,-1.907303,0.116018,-1.068429,-0.169181,0.765449,-0.428248,0.000054
1,-0.258666,0.347734,1.750804,0.744646,0.156363,0.059703,0.606871,1.274236,-0.051907,-0.414460,-0.900400,-1.083901,0.238065,0.040403,0.185373,0.233348,-0.223842,-0.417781,0.972869,-0.270457,-0.038910,-1.044466,0.511643,-0.243701,0.872497,-0.779433,0.340661,0.380800,-0.721053,0.327722,-0.943484,0.801839,0.370747,-0.736390,0.492543,1.436235,-1.465481,0.170475,-0.266959,-0.868216,...,-0.946630,0.280665,0.495754,-0.412071,0.190216,-0.197012,-0.586232,-0.148826,0.311156,-0.298051,-0.389379,-1.686341,-1.592549,0.704259,0.778034,0.850522,0.285663,0.283842,-0.357595,-0.127402,-0.374123,-0.541077,0.436833,0.087912,-0.209567,1.176815,-0.177077,0.422629,-0.575461,-0.475836,-0.815868,-0.885349,-0.129828,0.342767,-0.074613,-0.007231,0.095131,-0.297645,-0.237426,0.405776
2,0.068400,0.183420,-0.214393,0.162653,0.577132,0.442029,0.764438,-0.231416,0.047675,0.447656,0.244569,-0.412664,0.697799,0.426231,-0.888598,-0.014143,-0.230498,0.307927,0.393809,-0.087685,0.218682,-0.956841,0.148737,-0.326993,0.546378,1.332277,-0.055264,-0.255176,-1.306164,0.654879,-0.712077,0.094668,0.096023,-0.113867,0.053256,0.106918,-0.112052,0.227721,-0.690115,1.005142,...,-0.457578,-0.601099,-0.189601,-0.019634,-0.527271,-0.348555,0.013853,0.217463,0.498457,-0.234963,-0.239371,-0.890102,-0.499332,0.441001,0.115897,-1.437539,-0.679651,-0.743852,0.168532,-0.309895,0.265426,0.508672,0.518088,0.438903,0.816456,1.100221,-0.492533,-0.254384,-0.858149,-0.775391,-0.754896,-0.106961,-0.813766,-0.573372,0.283298,-1.146841,-0.188256,-0.448757,-0.169211,-0.402393
3,0.188355,0.749234,1.241174,-0.022515,-0.279121,-0.386572,1.099895,0.240622,0.614666,-0.238493,-0.190334,0.773524,0.447755,0.075109,0.457645,0.740499,-0.564793,-0.748223,-0.005571,-0.381286,0.149510,-0.817235,0.164348,0.037957,0.652394,0.676520,-0.199341,-0.201901,-1.347610,0.141949,0.251515,-0.318091,0.188833,-0.410054,0.170793,0.711227,0.472304,-0.038543,0.390866,-0.200143,...,-0.406574,-0.597680,0.496792,-0.099677,0.001599,-0.403225,0.212363,0.529139,0.696017,-0.658609,-0.574746,0.728369,0.420841,-0.083178,0.094958,-0.814671,-0.469056,0.236358,-0.311130,0.489052,0.181727,-0.319935,-0.147488,-0.030966,-0.183563,0.614271,0.485601,0.107692,-0.957782,-0.018880,0.304408,-0.002369,-0.649310,-2.455173,0.253222,-0.307889,-0.088449,-0.014566,0.152551,0.669754
4,0.212814,0.028354,2.198674,-0.266557,-0.021838,1.170687,0.923018,0.613718,0.008676,0.054019,-1.087324,0.586673,0.272260,0.487963,1.665598,0.155764,-0.577824,-0.609392,-0.021663,-0.091208,-0.054415,0.226796,0.252040,-1.596501,-0.688963,-1.063881,0.012446,-1.601381,-0.698446,0.426477,0.175979,0.685706,0.238747,0.318971,0.546566,0.163142,0.050185,-0.311229,-0.150162,-0.189366,...,-0.456599,-0.937988,-0.446278,0.615909,-0.336407,-0.202670,0.917652,0.180642,0.416239,-0.169856,-0.051804,0.151931,0.108152,0.935536,0.756829,0.140368,0.269411,-0.458251,-0.179566,0.204512,-0.525036,0.551151,1.038969,0.900104,0.064971,-0.623618,-0.379829,-0.745131,-0.930456,-0.3

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_embedding , y, test_size = 0.20, random_state = 0)

In [ ]:
# Fitting Kernel SVM to the Training set   (Accuracy = 80)
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0, verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Covid19/RandomForest_classifier_600.sav'
pickle.dump(classifier, open(filename, 'wb'))
print('Model saved')

Model saved


In [ ]:
# load the model from disk
filename = '/content/drive/MyDrive/Covid19/NB_classifier.sav'
classifier = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)
print('embeddings loaded')

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
print('y_pred is: ', y_pred)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('The confusion matrix is: ', cm)

Accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
print('The Accuracy is: ', Accuracy)

y_pred is:  [0 0 4 ... 0 0 0]
The confusion matrix is:  [[51008 12985]
 [14552 49267]]
The Accuracy is:  0.7845507464087879


In [ ]:
 #Training the K-NN model on the Training set   
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 12, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=12, p=2,
                     weights='uniform')

In [ ]:
# Training the Naive Bayes model on the Training set (Accuracy = 76)
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:

# Training the Logistic Regression model on the Training set (Accuracy = 80.11)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0,max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# Training the Random Forest Classification model on the Training set  (Accuracy = 78.4)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 30, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)